# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 112kB 13.9MB/s eta 0:00:01
Libraries imported.


In [2]:
!pip install lxml

     |████████████████████████████████| 5.5MB 5.3MB/s eta 0:00:01


In [3]:
!pip install html5lib

In [4]:
!pip install bs4

     |████████████████████████████████| 122kB 6.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


Load and explore the data

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [6]:
toronto_df = pd.read_html(url, header=0)[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
toronto_df.shape

(180, 3)

In [8]:
toronto_df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

In [9]:
toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [10]:
toronto_df.drop(toronto_df.index[toronto_df['Borough'] == 'Not assigned'], inplace=True)
toronto_df=toronto_df.reset_index(drop=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
toronto_dfPostalCode = toronto_df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_dfPostalCode.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### In this portion of the Project I imported all the necessary libraries required to proceed with the project such as Panda, Numpy, Folium, Cluster, Geopy, Matplotlib.  Then I installed all the packages I need inorder to fetch the data of the neighborhood of Toronto from wiki page such as lxml, html5lib, Bs4(Beautiful Soup4).   I did data acquisition from URL and then use the Pandas method read_html() to load the data from the web address.  Further, I used the method head() to display the first five rows of the dataframe. I listed the data types for each column. Info() function is being used as well to print a concise summary of a DataFrame.  In the last I grouped the dataframe as per Postal Code and Borough. In the below cell I am using shape method to print the number of rows of new dataframe.  

In [12]:
toronto_dfPostalCode.shape

(103, 3)

# Part 2

In [13]:
url= 'http://cocl.us/Geospatial_data'

In [14]:
df1 = pd.read_csv(url)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
toronto_data = toronto_dfPostalCode.join(df1.set_index('Postal Code'), on='Postal Code')
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# In the second part of this Project, I imported Geospatial dataset from CSV file, and used panda dataframe to read and load the data. I joined 2 dataframe together on Postal Code to associate the values of Longitude and Latitude with their respective Neighbourhood. 

# Part 3

In [17]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, toronto_data in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(toronto_data, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [20]:
CLIENT_ID = 'YLFCWQ3ZYWMALIKXC43K400R3U4O3CHZ3ASN2STT5HTJBTDY' # your Foursquare ID
CLIENT_SECRET = '0IJKUQ3A1CMH1Q5XBMG0SQ5M5Z1MHWSOVD2ZZXGUGZM00FRU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YLFCWQ3ZYWMALIKXC43K400R3U4O3CHZ3ASN2STT5HTJBTDY
CLIENT_SECRET:0IJKUQ3A1CMH1Q5XBMG0SQ5M5Z1MHWSOVD2ZZXGUGZM00FRU


In [21]:
toronto_data.head()

AttributeError: 'str' object has no attribute 'head'

In [24]:
print(toronto_data['Borough'])

TypeError: string indices must be integers

In [27]:
northyork_data = toronto_data.drop(toronto_data[toronto_data['Borough'] != 'North York'].index)
northyork_data.head()

AttributeError: 'str' object has no attribute 'drop'

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in North York. So let's slice the original dataframe and create a new dataframe of the North York data.

In [28]:
northyork_data = toronto_data[toronto_data['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

TypeError: string indices must be integers

In [29]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [36]:
# create map of Manhattan using latitude and longitude values
map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(North York['Latitude'], North York['Longitude'], North York['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_NorthYork

NameError: name 'NorthYork' is not defined

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.